In [1]:
import os

import random
import numpy as np
import pandas as pd

from transformers import AutoTokenizer

# from transformers import XLMRobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import (
    AutoModelForSequenceClassification,
    XLMRobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)


import torch
import torch.nn as nn
from torch.optim import AdamW, Adam
from torch.utils.data import Dataset, DataLoader

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 1
model_checkpoint = "distilbert-base-uncased-full-training"
model_name = "distilbert-base-uncased"
tokenizer_name = "distilbert-base-uncased"

In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained(
#     f"{model_checkpoint}/checkpoint-2589", num_labels=2
# )

config.json: 100%|██████████| 483/483 [00:00<00:00, 882kB/s]
model.safetensors: 100%|██████████| 268M/268M [00:24<00:00, 10.9MB/s] 
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 55.0kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.02MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.23MB/s]


In [5]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [6]:
def freeze_params(model):
    for name, param in model.named_parameters():
        if not name.startswith("classifier"):
            param.requires_grad = False

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [8]:
seed_everything(1)

In [9]:
# freeze_params(model)
if torch.cuda.is_available():
    model = model.to("cuda")

In [10]:
train = pd.read_csv("data/original/train.csv")
test = pd.read_csv("data/original/test.csv")
valid = pd.read_csv("data/original/validation.csv")

In [11]:
mapping = {"human": 0, "bot": 1}

In [13]:
train["label"] = train["account.type"].apply(lambda x: mapping[x])
test["label"] = test["account.type"].apply(lambda x: mapping[x])
valid["label"] = valid["account.type"].apply(lambda x: mapping[x])

In [14]:
class TweetDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data = self.data.iloc[index]

        text = data.text
        label = data.label

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt",
        )

        return dict(
            text=text,
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            label=torch.tensor(label, dtype=torch.long),
        )

In [16]:
batch_size = 8
lr = 2e-6

In [17]:
ds_train = TweetDataset(train, tokenizer=tokenizer, max_len=512)
ds_test = TweetDataset(test, tokenizer=tokenizer, max_len=512)
ds_valid = TweetDataset(valid, tokenizer=tokenizer, max_len=512)

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).astype(np.float32).mean().item()}

In [20]:
args = TrainingArguments(
    f"{model_checkpoint}",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,  # save best and last
    load_best_model_at_end=True,
    seed=seed,
    # metric_for_best_model=metric_name,
)

In [22]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(AdamW(model.parameters(), lr=lr), None),  # Optimizer, Scheduler
    # compute_loss to override one must create cutsom trainer class and override the method compute_loss(self, model, inputs, return_outputs=False):
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

In [23]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.331100,0.351013,0.839270
2,0.289800,0.316780,0.861859
3,0.282400,0.310971,0.866638
4,0.252600,0.332558,0.870982
5,0.253400,0.361340,0.875326


TrainOutput(global_step=12945, training_loss=0.296605224137879, metrics={'train_runtime': 3224.5527, 'train_samples_per_second': 64.232, 'train_steps_per_second': 8.029, 'total_flos': 1.371832380481536e+16, 'train_loss': 0.296605224137879, 'epoch': 5.0})

In [24]:
test_results = trainer.predict(ds_test)

In [25]:
test_results.metrics

{'test_loss': 0.3102383017539978,
 'test_accuracy': 0.8666927218437195,
 'test_runtime': 23.9829,
 'test_samples_per_second': 106.659,
 'test_steps_per_second': 13.343}

In [26]:
from sklearn.metrics import (
    balanced_accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)


def proba_to_pred(proba):
    pred = (proba > 0.5).astype(int)
    return pred


def calculate_metrics(y_true, y_pred):
    results = {
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
        "f1_score": f1_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
    }
    return results

In [27]:
calculate_metrics(test.label.values, np.argmax(test_results.predictions, axis=1))

{'balanced_accuracy': 0.8666880624021909,
 'f1_score': 0.8675728155339807,
 'precision': 0.8625482625482626,
 'recall': 0.87265625}